In [6]:
import pandas as pd
from code.gru.gru import gru
from code.parser.parse import Parser
from code.gru.gru import GRUModel
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, confusion_matrix
from code.wiener_kolmogorov.wiener_kolmogorov import WienerKolmogorov as WK
from code.detector.detector import  Detector
import numpy as np
from tensorflow import keras
from keras.utils.vis_utils import plot_model
from sklearn.model_selection import train_test_split
from keras.models import load_model

In [3]:
!pip install visualkeras

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 994.6/994.6 KB 5.5 MB/s eta 0:00:00a 0:00:01
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [7]:
def rates(true, pre, positives, negatives):
    tn, fp, fn, tp = confusion_matrix(true, pre).ravel()
    return accuracy_score(true, pre), tp / positives, tn / negatives

In [8]:
def lstm_data_transform(x_data, y_data, num_steps=5):
    X, y = list(), list()
    for i in range(x_data.shape[0]):
        end_ix = i + num_steps
        if end_ix >= x_data.shape[0]:
            break
        seq_X = x_data[i:end_ix]
        seq_y = y_data[end_ix]
        X.append(seq_X)
        y.append(seq_y)
    x_array = np.array(X)
    y_array = np.array(y)
    return x_array, y_array

In [9]:
def clean(preds):
    res= []
    for pred in preds:
        d0 = abs(pred - 0)
        d1 = abs(pred - 1)
        if d0 < d1:
            res.append(0)
        else:
            res.append(1)
    return res

In [10]:

df = pd.read_csv('../../runs/3/home.csv')
df = df[:2100]
df = df[["RSSI"]]

window = 300
zeros = np.zeros(window)
ones = np.ones(window)
y = ones
l = len(ones) + window
f = True
while l <= 2100:
    if f:
        y = np.concatenate((y, zeros))
        f = False
    else:
        y = np.concatenate((y, ones))
        f = True
    l += window
num_steps = 500


X, y = lstm_data_transform(df["RSSI"].to_numpy(), y, num_steps=num_steps)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.285)

from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import LSTM, GRU, SimpleRNN, Activation
from keras.layers import Dropout
from keras.regularizers import l2
import numpy as np
from keras import optimizers

model = Sequential()
model.add(GRU(200, activation='tanh', input_shape=(num_steps, 1),
               return_sequences=False))
model.add(Dense(units=50, activation='relu'))
model.add(Dense(units=1, activation='linear'))
model.compile(optimizer="adam", loss='mse')

In [16]:
model.save("local.h5")

In [12]:
model.fit(X_train, y_train, epochs=35)

Epoch 1/35
36/36 [==============================] - 15s 386ms/step - loss: 0.8789
Epoch 2/35
36/36 [==============================] - 14s 390ms/step - loss: 0.1745
Epoch 3/35
36/36 [==============================] - 14s 376ms/step - loss: 0.1551
Epoch 4/35
36/36 [==============================] - 14s 376ms/step - loss: 0.1354
Epoch 5/35
36/36 [==============================] - 14s 375ms/step - loss: 0.1271
Epoch 6/35
36/36 [==============================] - 14s 377ms/step - loss: 0.1331
Epoch 7/35
36/36 [==============================] - 14s 377ms/step - loss: 0.1233
Epoch 8/35
36/36 [==============================] - 14s 378ms/step - loss: 0.1124
Epoch 9/35
36/36 [==============================] - 14s 388ms/step - loss: 0.1102
Epoch 10/35
36/36 [==============================] - 15s 410ms/step - loss: 0.1059
Epoch 11/35
36/36 [==============================] - 14s 385ms/step - loss: 0.0944
Epoch 12/35
36/36 [==============================] - 14s 382ms/step - loss: 0.0815
Epoch 13/35
3

In [17]:
model.fit(X_train, y_train, epochs=1)

36/36 [==============================] - 13s 374ms/step - loss: 0.0572


In [3]:
saved = load_model("./models/home2")

In [18]:
cleaned = clean(model.predict(X_test))
accuracy_score(y_test, cleaned)

0.918859649122807

In [21]:
saved.fit(X_train, y_train, epochs=1)

36/36 [==============================] - 15s 374ms/step - loss: 0.0519


In [24]:
model.fit(X_train, y_train, epochs=1)

36/36 [==============================] - 14s 379ms/step - loss: 0.0455


In [30]:
cleaned = clean(model.predict(X_test))
accuracy_score(y_test, cleaned)

0.9912280701754386

In [26]:
model.save("./models/home2")

INFO:tensorflow:Assets written to: ./models/home2/assets


INFO:tensorflow:Assets written to: ./models/home2/assets


In [14]:
cleaned = clean(saved.predict(X_test))
acc, tpr, tnr = rates(y_test, cleaned, sum(y_test), len(cleaned) - sum(y_test))
print(acc, tpr, tnr)

0.9802631578947368 0.9703703703703703 0.9946236559139785
